In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [2]:
!pip install tensorflow==2.3.0
!pip install tensorflow-datasets==3.2.1
!pip install Pillow==7.2.0

In [1]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
      # Your Code
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('acc') > DESIRED_ACCURACY:
            self.model.stop_training = True

callbacks = myCallback()

--2020-12-25 15:50:34--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.9.208, 172.217.5.240, 172.217.15.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.9.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2020-12-25 15:50:34 (185 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [2]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
# Your Code Here
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [3]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip=True)# Your Code Here

train_generator = train_datagen.flow_from_directory(
                                  "/tmp/h-or-s",
                                  target_size=(150, 150),
                                  batch_size=4,
                                  class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [4]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
      train_generator,
      steps_per_epoch=20, 
      epochs = 100)
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/100
20/20 [==============================] - 0s 24ms/step - loss: 0.7145 - acc: 0.5000
Epoch 2/100
20/20 [==============================] - 1s 26ms/step - loss: 0.5712 - acc: 0.6875
Epoch 3/100
20/20 [==============================] - 0s 25ms/step - loss: 0.4846 - acc: 0.8000
Epoch 4/100
20/20 [==============================] - 1s 27ms/step - loss: 0.4093 - acc: 0.8000
Epoch 5/100
20/20 [==============================] - 1s 26ms/step - loss: 0.4129 - acc: 0.7875
Epoch 6/100
20/20 [==============================] - 1s 26ms/step - loss: 0.4342 - acc: 0.7625
Epoch 7/100
20/20 [==============================] - 1s 26ms/step - loss: 0.3247 - acc: 0.8625
Epoch 8/100
20/20 [==============================] - 1s 27ms/step - loss: 0.2494 - acc: 0.8625
Epoch 9/100
20/20 [==============================] - 1s 26ms/step - loss: 0.5785 - acc: 0.7875
Epoch 10/100
20/20 [==============================] - 1s 26ms/step - loss: 0.3856 - acc: 0.8500
Epoch 11/100
20/20 [=============================

In [5]:
model.save("model_week4.h5")